<a href="https://colab.research.google.com/github/Anoop-ai-1993/ACI-assignment/blob/main/ACI_Assignment_2_Solution_S2_121_Atul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Patched Catch-Up Notebook

This notebook contains a corrected implementation of the Catch-Up game with proper move generation and minimax + alpha-beta pruning. The `CatchUpGame` class now enumerates legal multi-number turns per the rules and uses a cached minimax search.

In [8]:
import sys
from functools import lru_cache
from math import inf
import copy

class CatchUpGame:
    """
    Corrected Catch-Up game implementation with support for multi-number turns
    and minimax + alpha-beta pruning.
    """

    def __init__(self, n):
        # Cap n at 20 for practical performance with minimax search
        self.n = min(int(n), 20)
        if n > 20:
            print("Warning: n capped at 20 for performance in optimal search.")
        # Use tuple for remaining numbers for immutability, essential for caching in minimax
        self.remaining = tuple(range(1, self.n + 1))
        self.p1_sum = 0
        self.p2_sum = 0
        self.first_move_done = False  # Flag for P1's special first move
        # This will store the total sum of the player who *just completed* their turn.
        # This becomes the target for the *next* player.
        self.previous_player_turn_sum = 0

    def copy(self):
        """Creates a deep copy of the current game state."""
        g = CatchUpGame(self.n)
        g.remaining = tuple(self.remaining) # Tuples are immutable, so direct assignment is fine
        g.p1_sum = self.p1_sum
        g.p2_sum = self.p2_sum
        g.first_move_done = self.first_move_done
        g.previous_player_turn_sum = self.previous_player_turn_sum
        return g

    def make_move(self, pick, is_p1_move):
        """
        Applies a 'pick' (a tuple of numbers) as a move.
        This method updates player sums and remaining numbers.
        It assumes 'pick' is a legal move.
        """
        if not pick:
            raise ValueError("An empty selection is not allowed.")

        rem_set = set(self.remaining)
        # Check if all numbers in the pick are actually available
        if any(x not in rem_set for x in pick):
            raise ValueError("One or more numbers in your pick are not available.")

        total_picked_sum_in_this_turn = sum(pick)

        if not self.first_move_done:
            # Rule 2: P1's first move must be a single number.
            if len(pick) != 1:
                raise ValueError("For the first move, P1 must choose exactly one number.")
            self.p1_sum += total_picked_sum_in_this_turn
            self.first_move_done = True
            # After P1's first move, P1's sum IS the previous turn sum for the next player (P2)
            self.previous_player_turn_sum = self.p1_sum
        else:
            # For subsequent moves (Rule 3)
            if is_p1_move:
                self.p1_sum += total_picked_sum_in_this_turn
                # After P1 moves, P1's new sum becomes the previous turn sum for P2's next turn
                self.previous_player_turn_sum = self.p1_sum
            else:
                self.p2_sum += total_picked_sum_in_this_turn
                # After P2 moves, P2's new sum becomes the previous turn sum for P1's next turn
                self.previous_player_turn_sum = self.p2_sum

        # Remove picked numbers from the remaining set
        new_rem = list(self.remaining)
        for x in pick:
            new_rem.remove(x)
        self.remaining = tuple(sorted(new_rem)) # Keep remaining sorted for consistency


    def is_terminal(self):
        """Checks if the game has ended (no numbers left)."""
        return len(self.remaining) == 0

    def evaluate_final(self):
        """
        Evaluates the game state from P1's perspective.
        P1 wants to maximize (P1's sum - P2's sum).
        """
        return self.p1_sum - self.p2_sum

    def _generate_subsets_to_reach(self, target_sum_for_turn, current_remaining_numbers, current_player_base_sum):
        """
        Generates subsets of 'current_remaining_numbers' whose sum, when added to
        'current_player_base_sum', meets or exceeds 'target_sum_for_turn'.
        This handles Rule 3: "sum of its choices up to that point equals or just exceeds
        its opponent's previous sum."
        """
        if not current_remaining_numbers:
            return

        rem_sorted = sorted(list(current_remaining_numbers), reverse=True)
        n_rem = len(rem_sorted)

        found_any_valid_subset = False

        # This is the 'sum of choices' needed from the pick itself
        required_sum_in_this_pick = max(0, target_sum_for_turn - current_player_base_sum)

        def backtrack(index, current_pick_sum, current_subset):
            nonlocal found_any_valid_subset

            if current_pick_sum >= required_sum_in_this_pick and current_subset:
                found_any_valid_subset = True
                yield tuple(sorted(current_subset))
                return

            if index >= n_rem:
                return

            # Option 1: Include the current number (rem_sorted[index])
            val = rem_sorted[index]
            yield from backtrack(index + 1, current_pick_sum + val, current_subset + [val])

            # Option 2: Exclude the current number (rem_sorted[index])
            yield from backtrack(index + 1, current_pick_sum, current_subset)

        generated_subsets = set()
        for subset in backtrack(0, 0, []):
            generated_subsets.add(subset)
            yield subset

        # Endgame Fallback (Rule 3 ambiguity resolution):
        # If no subset was found that, when added to the player's base sum,
        # meets or exceeds the target, the player must take ALL remaining numbers.
        if not found_any_valid_subset and current_remaining_numbers:
            all_remaining_as_tuple = tuple(current_remaining_numbers)
            if all_remaining_as_tuple not in generated_subsets:
                yield all_remaining_as_tuple


    def legal_turns(self, is_p1_turn):
        """
        Generates all legal moves (each move is a tuple of numbers) for the current state
        and current player, adhering to the game rules.
        """
        if not self.remaining: # If no numbers left, no legal moves.
            return

        if not self.first_move_done:
            # Rule 2: P1's first move allows any single remaining number.
            if is_p1_turn: # Only P1 makes the very first move
                for x in self.remaining:
                    yield (x,)
            return

        # Determine the target for the current player's turn (Rule 3).
        # This is the sum of the opponent's previous complete turn.
        # This is stored in `self.previous_player_turn_sum`
        target_sum_for_turn = self.previous_player_turn_sum

        # The current player's total sum *before* this turn
        current_player_base_sum = self.p1_sum if is_p1_turn else self.p2_sum

        # Generate subsets of remaining numbers that satisfy the "catch-up" rule.
        for subset in self._generate_subsets_to_reach(target_sum_for_turn, self.remaining, current_player_base_sum):
            if subset:
                yield subset

    _memo = {}

    def minimax(self, is_maximizing_player, alpha=-inf, beta=inf):
        """
        Implements the Minimax algorithm with Alpha-Beta pruning to find the optimal move
        for the current game state.
        Returns the utility (score from P1's perspective) of the best possible play.
        """
        if self.is_terminal():
            return self.evaluate_final()

        key = (self.remaining, self.p1_sum, self.p2_sum, self.first_move_done, self.previous_player_turn_sum, is_maximizing_player)
        if key in self._memo:
            return self._memo[key]

        if is_maximizing_player: # P1's turn (Maximizing player)
            value = -inf
            for move in self.legal_turns(is_p1_turn=True):
                game_copy = self.copy()
                try:
                    game_copy.make_move(move, is_p1_move=True)
                except ValueError:
                    continue

                val = game_copy.minimax(False, alpha, beta)
                value = max(value, val)
                alpha = max(alpha, value)

                if beta <= alpha:
                    break
            self._memo[key] = value
            return value
        else: # P2's turn (Minimizing player)
            value = inf
            for move in self.legal_turns(is_p1_turn=False):
                game_copy = self.copy()
                try:
                    game_copy.make_move(move, is_p1_move=False)
                except ValueError:
                    continue

                val = game_copy.minimax(True, alpha, beta)
                value = min(value, val)
                beta = min(beta, value)

                if beta <= alpha:
                    break
            self._memo[key] = value
            return value

    def best_move(self, is_p1_turn=True):
        """
        Determines the optimal move for the current player (P1 or P2)
        using the minimax algorithm with alpha-beta pruning.
        Returns the best move (tuple of numbers) and its evaluated value.
        """
        best_move_found = None

        # Clear memoization cache for a fresh search from the current state
        CatchUpGame._memo = {}

        if is_p1_turn:
            best_value = -inf
            for m in self.legal_turns(is_p1_turn=True):
                g_copy = self.copy()
                try:
                    g_copy.make_move(m, is_p1_move=True)
                except ValueError:
                    continue

                val = g_copy.minimax(False, -inf, inf) # P2 will minimize next
                if val > best_value:
                    best_value = val
                    best_move_found = m
        else: # P2's turn
            best_value = inf
            for m in self.legal_turns(is_p1_turn=False):
                g_copy = self.copy()
                try:
                    g_copy.make_move(m, is_p1_move=False)
                except ValueError:
                    continue

                val = g_copy.minimax(True, -inf, inf) # P1 will maximize next
                if val < best_value:
                    best_value = val
                    best_move_found = m
        return best_move_found, best_value

def get_validated_input(prompt, type_converter=int, valid_range=None, allow_multi_numbers=False):
    """
    Helper function to get validated input from the user.
    Handles type conversion, range checking, and multiple space-separated numbers.
    """
    while True:
        try:
            user_input = input(prompt).strip()
            if not user_input:
                print("Input cannot be empty. Please try again.")
                continue

            if allow_multi_numbers:
                numbers_str = user_input.split()
                if not numbers_str:
                    print("Please enter at least one number.")
                    continue
                values = tuple(type_converter(s) for s in numbers_str)
                if any(v <= 0 for v in values):
                    print("All numbers must be positive natural numbers.")
                    continue
            else:
                values = type_converter(user_input)
                if values <= 0:
                    print("Input must be a positive natural number.")
                    continue

            if not allow_multi_numbers and valid_range:
                min_val, max_val = valid_range
                if not (min_val <= values <= max_val):
                    print(f"Please enter a value between {min_val} and {max_val}.")
                    continue

            return values
        except ValueError:
            print("Invalid input. Please enter a natural number (or space-separated natural numbers).")
        except Exception as e:
            print(f"An unexpected error occurred: {e}. Please try again.")

def play_game_interactive():
    """
    Interactive game loop for Catch-Up, allowing two human players.
    P1 and P2 take turns, making choices dynamically.
    """
    print("\n=== Welcome to Catch-Up! ===")
    print("In this game, two players pick numbers from a set to get a higher total sum.")
    print("The highest number in the set (n) can be between 2 and 20 for optimal play.")

    n = get_validated_input("Enter the highest number (n) for the set (e.g., 7): ", valid_range=(2, 20))
    game = CatchUpGame(n)

    print(f"\nGame initialized with numbers from 1 to {n}.")
    print("--- Game Rules Recap ---")
    print("1. P1 makes the first move, choosing only one number.")
    print("2. Thereafter, players choose one or more numbers.")
    print("3. A turn ends when the sum of numbers chosen in that turn, plus your current score,")
    print("   equals or just exceeds your opponent's previous total score.")
    print("4. If no combination meets the rule, you take all remaining numbers.")
    print("5. Goal: Have a higher sum than your opponent at the end!")
    print("------------------------")

    turn_counter = 0

    while not game.is_terminal():
        is_p1_turn = (turn_counter % 2 == 0)
        current_player_name = "P1" if is_p1_turn else "P2"

        print(f"\n--- Turn {turn_counter + 1} ---")
        print(f"Available numbers: {game.remaining}")
        print(f"P1 Total Score: {game.p1_sum} | P2 Total Score: {game.p2_sum}")

        # Display the target for the current turn, which is the opponent's previous turn's sum.
        if game.first_move_done:
            # The target sum for the current player's *entire score* must be >= `game.previous_player_turn_sum`.
            # The `previous_player_turn_sum` stores the total sum achieved by the *last player* who moved.
            current_player_current_score = game.p1_sum if is_p1_turn else game.p2_sum

            # The amount that needs to be added by the current player in this turn
            # to meet or exceed the `previous_player_turn_sum`.
            required_sum_from_pick = max(0, game.previous_player_turn_sum - current_player_current_score)

            print(f"{current_player_name}, your target is to have a total score of at least {game.previous_player_turn_sum} at the end of this turn.")
            print(f"You currently have {current_player_current_score}. You need to pick numbers that sum to at least {required_sum_from_pick} in this turn.")
        else:
            print("P1, it's your first move. You must pick exactly ONE number (no specific target sum yet).")

        while True:
            try:
                if not game.first_move_done:
                    user_pick = get_validated_input(f"{current_player_name}, pick ONE number: ", allow_multi_numbers=False)
                    player_move_tuple = (user_pick,)
                else:
                    user_pick_str = get_validated_input(
                        f"{current_player_name}, enter numbers to pick (space-separated, e.g., '2 4 6'): ",
                        allow_multi_numbers=True
                    )
                    player_move_tuple = user_pick_str

                is_legal_move = False
                for legal_m in game.legal_turns(is_p1_turn):
                    if player_move_tuple == legal_m:
                        is_legal_move = True
                        break

                if not is_legal_move:
                    print("That's an invalid move according to the game rules! Please pick numbers that correctly 'catch up' or follow the first move rule.")
                    if game.first_move_done:
                        current_player_current_score = game.p1_sum if is_p1_turn else game.p2_sum
                        required_sum_from_pick = max(0, game.previous_player_turn_sum - current_player_current_score)
                        current_pick_sum = sum(player_move_tuple)
                        if current_pick_sum < required_sum_from_pick:
                            print(f"The sum of your chosen numbers ({current_pick_sum}) is not enough. You need to pick numbers that sum to at least {required_sum_from_pick} in this turn.")
                        elif any(n not in game.remaining for n in player_move_tuple):
                            print("One or more of the numbers you tried to pick are not available.")
                        else:
                             print("The combination of numbers chosen is not a valid way to achieve the target or is not the set of all remaining numbers if no other subset works.")
                    continue

                game.make_move(player_move_tuple, is_p1_move=is_p1_turn)
                print(f"{current_player_name} picked: {player_move_tuple}")
                break
            except ValueError as e:
                print(f"Error: {e}. Please try again.")
            except Exception as e:
                print(f"An unexpected error occurred during move: {e}. Please report this.")
                sys.exit(1)

        turn_counter += 1

    print("\n=== Game Over! ===")
    print(f"Final Scores - P1: {game.p1_sum} | P2: {game.p2_sum}")

    if game.p1_sum > game.p2_sum:
        print("🎉 P1 wins! Congratulations! 🎉")
    elif game.p2_sum > game.p1_sum:
        print("🎉 P2 wins! Congratulations! 🎉")
    else:
        print("🤝 It's a tie! Well played by both! 🤝")

if __name__ == "__main__":
    play_game_interactive()


=== Welcome to Catch-Up! ===
In this game, two players pick numbers from a set to get a higher total sum.
The highest number in the set (n) can be between 2 and 20 for optimal play.
Enter the highest number (n) for the set (e.g., 7): 10

Game initialized with numbers from 1 to 10.
--- Game Rules Recap ---
1. P1 makes the first move, choosing only one number.
2. Thereafter, players choose one or more numbers.
3. A turn ends when the sum of numbers chosen in that turn, plus your current score,
   equals or just exceeds your opponent's previous total score.
4. If no combination meets the rule, you take all remaining numbers.
5. Goal: Have a higher sum than your opponent at the end!
------------------------

--- Turn 1 ---
Available numbers: (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
P1 Total Score: 0 | P2 Total Score: 0
P1, it's your first move. You must pick exactly ONE number (no specific target sum yet).
P1, pick ONE number: 6
P1 picked: (6,)

--- Turn 2 ---
Available numbers: (1, 2, 3, 4, 5, 7,

KeyboardInterrupt: Interrupted by user

In [3]:

import unittest

class TestCatchUpGame(unittest.TestCase):
    def test_first_move_single_number(self):
        g = CatchUpGame(5)
        moves = list(g.legal_turns())
        self.assertTrue(all(len(m)==1 for m in moves))

    def test_illegal_first_move_multiple_numbers(self):
        g = CatchUpGame(5)
        with self.assertRaises(ValueError):
            g.make_move((1,2))

    def test_terminal_state(self):
        g = CatchUpGame(1)
        g.make_move((1,))
        self.assertTrue(g.is_terminal())

    def test_evaluation_margin(self):
        g = CatchUpGame(3)
        g.p1_sum, g.p2_sum = 5, 2
        self.assertEqual(g.evaluate_final(), 3)

    def test_tie_evaluation(self):
        g = CatchUpGame(3)
        g.p1_sum, g.p2_sum = 4, 4
        self.assertEqual(g.evaluate_final(), 0)

    def test_best_move_non_none(self):
        g = CatchUpGame(3)
        move, val = g.best_move(True)
        self.assertIsNotNone(move)

    def test_play_small_game(self):
        g = CatchUpGame(2)
        play_game(2, verbose=False)
        self.assertTrue(True)  # no crash

    def test_remaining_numbers_decrease(self):
        g = CatchUpGame(4)
        m = (1,)
        g.make_move(m)
        self.assertEqual(len(g.remaining), 3)

    def test_minimax_consistency(self):
        g = CatchUpGame(3)
        val1 = g.minimax(True)
        val2 = g.minimax(True)
        self.assertEqual(val1, val2)

    def test_best_move_improves_score(self):
        g = CatchUpGame(3)
        m, val = g.best_move(True)
        self.assertIsInstance(val, int)

    def test_invalid_pick_raises(self):
        g = CatchUpGame(3)
        with self.assertRaises(ValueError):
            g.make_move((99,))

    def test_copy_independence(self):
        g = CatchUpGame(3)
        h = g.copy()
        g.make_move((1,))
        self.assertNotEqual(len(g.remaining), len(h.remaining))

    def test_first_move_done_flag(self):
        g = CatchUpGame(3)
        g.make_move((1,))
        self.assertTrue(g.first_move_done)

    def test_legal_turns_nonempty(self):
        g = CatchUpGame(3)
        g.make_move((1,))
        moves = list(g.legal_turns())
        self.assertTrue(len(moves) > 0)

    def test_best_move_p2_turn(self):
        g = CatchUpGame(3)
        g.make_move((1,))
        m, val = g.best_move(False)
        self.assertIsNotNone(m)

    def test_tie_game_play(self):
        g = CatchUpGame(2)
        g.p1_sum, g.p2_sum = 3, 3
        self.assertEqual(g.evaluate_final(), 0)

    def test_large_n_cap(self):
        g = CatchUpGame(100)
        self.assertEqual(g.n, 20)

    def test_play_game_runs(self):
        play_game(3, verbose=False)
        self.assertTrue(True)

    def test_generate_subsets(self):
        g = CatchUpGame(4)
        g.first_move_done = True
        subs = list(g._generate_subsets_to_reach(3, (1,2,3,4)))
        self.assertTrue(any(sum(s)>=3 for s in subs))

    def test_full_game_simulation(self):
        play_game(5, verbose=False)
        self.assertTrue(True)


# Run unit tests
suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestCatchUpGame)
unittest.TextTestRunner(verbosity=2).run(suite)

# Automated 20-game simulation
print("\n=== Automated 20-game Simulation ===")
for n in range(3, 23):  # 20 games (3..22)
    print(f"\n--- Game n={n} ---")
    play_game(n, verbose=False)


test_best_move_improves_score (__main__.TestCatchUpGame.test_best_move_improves_score) ... ok
test_best_move_non_none (__main__.TestCatchUpGame.test_best_move_non_none) ... ok
test_best_move_p2_turn (__main__.TestCatchUpGame.test_best_move_p2_turn) ... ok
test_copy_independence (__main__.TestCatchUpGame.test_copy_independence) ... ok
test_evaluation_margin (__main__.TestCatchUpGame.test_evaluation_margin) ... ok
test_first_move_done_flag (__main__.TestCatchUpGame.test_first_move_done_flag) ... ok
test_first_move_single_number (__main__.TestCatchUpGame.test_first_move_single_number) ... ok
test_full_game_simulation (__main__.TestCatchUpGame.test_full_game_simulation) ... ok
test_generate_subsets (__main__.TestCatchUpGame.test_generate_subsets) ... ok
test_illegal_first_move_multiple_numbers (__main__.TestCatchUpGame.test_illegal_first_move_multiple_numbers) ... ok
test_invalid_pick_raises (__main__.TestCatchUpGame.test_invalid_pick_raises) ... ok
test_large_n_cap (__main__.TestCatchUpGa


=== Automated 20-game Simulation ===

--- Game n=3 ---

--- Game n=4 ---

--- Game n=5 ---

--- Game n=6 ---

--- Game n=7 ---

--- Game n=8 ---

--- Game n=9 ---

--- Game n=10 ---

--- Game n=11 ---

--- Game n=12 ---


KeyboardInterrupt: 